In [ ]:
import re
import time
import os
from dotenv import load_dotenv

from google import genai
from google.genai import types

In [2]:
load_dotenv('../../etc/.env')
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

In [16]:
# 1. 모든 스토어 목록 가져오기
print("--- 내 모든 File Search Store 목록 ---")
for store in client.file_search_stores.list():
    print(f"이름: {store.name}")
    print(f"표시 이름: {store.display_name}")
    print(f"생성 시간: {store.create_time}")
    print("-" * 30)

--- 내 모든 File Search Store 목록 ---
이름: fileSearchStores/jbriskmanual-01q1nf25k1tw
표시 이름: JB_RISK_MANUAL
생성 시간: 2026-01-29 08:22:12.725911+00:00
------------------------------


In [ ]:

# # 삭제하고 싶은 스토어의 ID (예: fileSearchStores/12345...)
# store_id = "fileSearchStores/jbragtest-4j26m066cjya"

# # 스토어와 내부 데이터 전체 삭제
# client.file_search_stores.delete(name=store_id, config={'force': True})

# print(f"스토어 {store_id}가 성공적으로 삭제되었습니다.")

스토어 fileSearchStores/jbragtest-4j26m066cjya가 성공적으로 삭제되었습니다.


In [15]:
file_search_store = client.file_search_stores.create(
    config={'display_name': 'JB_RISK_MANUAL'}
)

In [18]:
files_to_upload = [
    ('pdf/manual.pdf', '업무메뉴얼_v1.0'),
    ('pdf/tlgod.pdf', '시행세칙'),
    ('pdf/dyrjs.pdf', '업무요건정의서')
]

# 2. 반복문을 통해 업로드 진행
for file_path, display_name in files_to_upload:
    print(f"{display_name} 업로드 시작...")
    
    operation = client.file_search_stores.upload_to_file_search_store(
        file=file_path,
        file_search_store_name=file_search_store.name,
        config={
            'display_name': display_name,
            'chunking_config': {
                'white_space_config': {
                    'max_tokens_per_chunk': 200,
                    'max_overlap_tokens': 20
                }
            }
        }
    )
    # 처리 대기
    while not operation.done:
        time.sleep(5)
        operation = client.operations.get(operation)
    # result = operation.result() # 인덱싱이 끝날 때까지 여기서 대기합니다.
    # print(f"인덱싱 완료: {result}")
    
    # 업로드가 완료될 때까지 기다림 (선택 사항)
    # 파일이 아주 많다면 반복문 밖에서 한꺼번에 상태를 체크하는 것이 효율적입니다.
    # operation.result() 
    
    print(f"{display_name} 업로드 요청 완료.")

업무메뉴얼_v1.0 업로드 시작...
업무메뉴얼_v1.0 업로드 요청 완료.
시행세칙 업로드 시작...
시행세칙 업로드 요청 완료.
업무요건정의서 업로드 시작...
업무요건정의서 업로드 요청 완료.


In [32]:
# 질문을 던질 때 제약 조건을 줍니다.
response = client.models.generate_content(
    model="gemini-2.5-flash", # 혹은 사용 중인 모델
    contents="""
    신용리스크 표준방법 RW 설명해줘
    """,
    config=types.GenerateContentConfig(
        temperature=0.0,
        
        # 2. 시스템 인스트럭션 추가 (가장 강력한 제약)
        system_instruction="""
        너는 금융 규제 전문가야. 
        반드시 제공된 'File Search' 결과 내의 정보만을 바탕으로 답변해야 해.
        파일에 없는 수치나 내용은 절대 지어내지 말고, 
        만약 파일에서 찾을 수 없다면 '해당 정보는 문서에 포함되어 있지 않습니다'라고 대답해.
        """,

        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=["fileSearchStores/jbriskmanual-01q1nf25k1tw"]
                )
            )
        ]
    )
)

In [33]:
print(response.text)

신용리스크 표준방법(Standardized Approach)에서 위험가중치(RW, Risk Weight)는 거래상대방의 섹터(Sector)와 신용도에 따라 적용되는 값입니다. 이 RW는 표준방법에 따른 소요자기자본을 산출할 때 사용되며, 신뢰수준에 따른 승수는 적용되지 않습니다.

**RW 결정 방법 및 영향 요인:**

1.  **섹터 및 신용도에 따른 차등 적용:** RW는 거래상대방의 섹터와 신용도에 따라 다르게 적용됩니다. 이는 신용스프레드의 변동성을 반영하는 요소입니다.
2.  **적격 외부신용등급 활용:** 표준방법에서는 적격 외부신용평가기관(ECAI)이 평가한 신용등급을 통해 표준신용등급을 결정하고, 이에 따라 RW를 적용합니다. 국내 적격 ECAI로는 한국신용정보, 한국신용평가, 한국기업평가가 있으며, 해외 기관으로는 S&P, Moody's, Fitch가 있습니다.
3.  **다양한 RW 결정 유형:** 표준방법 적용 대상의 RW 결정 방법은 다음과 같은 9가지 유형으로 구분될 수 있습니다.
    *   **자산분류별 RW 결정 (SA-자산분류별 RW 결정):** 공공기관, 주식, 부동산개발금융, 소매, 부도, 기타자산 및 집합투자증권 중 채안펀드 등에 적용됩니다.
    *   **OECD 등급 적용 (SA-OECD 등급적용):** 정부 및 외국 정부에 적용됩니다.
    *   **ECAI 등급 적용 (SA-ECAI 등급적용):** 적격 외부신용등급을 보유한 은행, 기업, 특수금융에 적용됩니다.
    *   **실사등급 적용 (SA-실사등급적용):** 실사등급을 적용하는 은행에 해당합니다.
    *   **LTV 구간별 RW 결정 (SA-LTV 구간별 RW 결정):** 주거용/상업용 부동산의 LTV 비율에 따라 적용됩니다.
    *   **커버드본드 RW 결정 (SA-커버드본드 RW 결정):** 커버드본드에 적용되며, 등급 RW가 우선 적용되고 무등급 시 신용 RW가 적용됩니다.
    *   **집합투자증권 약정서기반 접근법 (SA-

In [ ]:
# 1. 출처를 저장할 집합(set) 생성
source_list = set()

# 2. 결과 메타데이터에서 청크(조각)들을 하나씩 확인
metadata = response.candidates[0].grounding_metadata

if metadata.grounding_chunks:
    for chunk in metadata.grounding_chunks:
        source = chunk.retrieved_context
        if source and source.text:
            title = source.title
            
            # [핵심] 텍스트 내에서 --- PAGE 200 --- 패턴 찾기
            # 정규표현식 설명: '--- PAGE' 다음에 '숫자(\d+)'가 오고 '---'로 닫히는 패턴
            page_match = re.search(r'--- PAGE (\d+) ---', source.text)
            
            if page_match:
                page_num = page_match.group(1)
                source_list.add(f"{title} ({page_num}페이지)")
            else:
                # 페이지 번호가 없을 경우 제목만 추가
                source_list.add(title)

print(response.text)
# 3. 예쁘게 출력
if source_list:
    # 정렬해서 보기 좋게 합치기
    final_sources = ", ".join(sorted(list(source_list)))
    print(f"📌 최종 출처: {final_sources}")
else:
    print("📌 출처를 찾을 수 없습니다.")

📌 최종 출처: 업무요건정의서 (238페이지), 업무요건정의서 (254페이지), 업무요건정의서 (309페이지), 업무요건정의서 (320페이지), 업무요건정의서 (326페이지)
